### Options Pricer

Saurav Luthra, Dec 2021

My goal is to calculate the price of options along a certain stock's option chain using the Black-Scholes Model, and eventually scrape real-time market data to compare real bid/ask prices to calculated prices. The stock whose options chain is modeled will be user selected, as well as the expiry date for the option chain.

I'm hoping this options pricer project will lead me to a more in-depth understanding of options pricing and volatility, as well as the technical tools needed to create an algorithmic trading strategy from scratch. This project will not account for transaction costs and will not be able to execute trades on its own (due to some external restrictions) however I hope to implement such features in the future.

In [31]:
from math import log, sqrt, pi, exp
import sys
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
import pandas_datareader as web
from math import *

In [35]:
def norm_cdf(x):
    #'Cumulative distribution function for the standard normal distribution'
    #https://stackoverflow.com/questions/809362/how-to-calculate-cumulative-normal-distribution
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def norm_pdf(x, mean, sd):
     #'Probability Density function for the standard normal distribution'
    #https://stackoverflow.com/questions/12412895/how-to-calculate-probability-in-a-normal-distribution-given-mean-standard-devi
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

In [28]:
def d1 (S,K,T,r,sigma):
    return(log(S/K) + (r + sigma**2/2.) * T) / (sigma * sqrt(T))

def d2 (S,K,T,r,sigma):
    return d1(S,K,T,r,sigma) - sigma * sqrt(T)

def bs_call (S,K,T,r,sigma):
    return S * norm_cdf(d1(S,K,T,r,sigma)) - K * exp(-r * T) * norm_cdf(d2(S,K,T,r,sigma))
  
def bs_put (S,K,T,r,sigma):
    return K * exp(-r * T) - S + bs_call(S,K,T,r,sigma)

In [25]:
stock = 'SPY'
expiry = '12-18-2023'
strike_price = 200

today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

print("Today, One Yr Ago:", today, one_year_ago)

Today, One Yr Ago: 2022-01-02 23:22:34.458453 2021-01-02 23:22:34.458453


In [30]:
df = web.DataReader(stock, 'yahoo', one_year_ago, today)

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = np.sqrt(252) * df['returns'].std()
uty = 0.015
#(web.DataReader("^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365

print('The Option Price is: $', bs_call(lcp, strike_price, t, uty, sigma))

The Option Price is:  280.7352458121694


In [32]:
df

,High,Low,Open,Close,Volume,Adj Close,close_day_before,returns
Date,,,,,,,,
2021-01-04,375.450012,364.820007,375.309998,368.790009,110210800.0,363.936432,NaN,NaN
2021-01-05,372.500000,368.049988,368.100006,371.329987,66426200.0,366.442993,368.790009,0.006887
2021-01-06,376.980011,369.119995,369.709991,373.549988,107997700.0,368.633789,371.329987,0.005979
2021-01-07,379.899994,375.910004,376.100006,379.100006,68766800.0,374.110748,373.549988,0.014857
2021-01-08,381.489990,377.100006,380.589996,381.260010,71677200.0,376.242371,379.100006,0.005698
...,...,...,...,...,...,...,...,...
2021-12-27,477.309998,472.010010,472.059998,477.260010,56808600.0,477.260010,470.600006,0.014152
2021-12-28,478.809998,476.059998,477.720001,476.869995,47274600.0,476.869995,477.260010,-0.000817
2021-12-29,478.559998,475.920013,476.980011,477.480011,54503000.0,477.480011,476.869995,0.001279


In [36]:
def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm_cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm_cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

print("Implied Volatility: " +
      str(100 * call_implied_volatility(bs_call(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")

Implied Volatility: 0.1 %


In [11]:
def call_delta(S,K,T,r,sigma):
    return norm_cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm_pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm_pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm_pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm_cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm_cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm_cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm_pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm_pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm_pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm_cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm_cdf(-d2(S,K,T,r,sigma)))